# Tensor Board and To reuse model

## To insert modules

In [1]:
import numpy as np
import tensorflow as tf

## Data setup

In [2]:
data=np.loadtxt('./data_c5.csv',delimiter=',',unpack=True,dtype='float32')
x_data=np.transpose(data[0:2])
y_data=np.transpose(data[2:])

## A sort of Variable

In [3]:
global_step=tf.Variable(0,trainable=False,name='global_step')

## Place holder setup

In [4]:
X=tf.placeholder(tf.float32)
Y=tf.placeholder(tf.float32)

## Variables setup

In [5]:
W1=tf.Variable(tf.random_uniform([2,10],-1.,1.))
L1=tf.nn.relu(tf.matmul(X,W1))
W2=tf.Variable(tf.random_uniform([10,20],-1.,1.))
L2=tf.nn.relu(tf.matmul(L1,W2))
W3=tf.Variable(tf.random_uniform([20,3],-1.,1.))
model=tf.matmul(L2,W3)

## Object function and Opimization class setup

In [6]:
cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y,logits=model))
optimizer=tf.train.AdamOptimizer(learning_rate=0.01)
train_op=optimizer.minimize(cost,global_step=global_step)

## Session setup

In [7]:
sess=tf.Session()
saver=tf.train.Saver(tf.global_variables())

## To check 'Check Point'

In [8]:
ckpt=tf.train.get_checkpoint_state('./model')
if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
    saver.restore(sess,ckpt.model_checkpoint_path)
else:
    sess.run(tf.global_variables_initializer())

INFO:tensorflow:Restoring parameters from ./model\dnn.ckpt-2


## Learning process

In [52]:
for step in range(2):
    sess.run(train_op,feed_dict={X:x_data,Y:y_data})
    
    print('Step: %d, Cost: %.3f' %(sess.run(global_step),sess.run(cost,feed_dict={X:x_data,Y:y_data})))

Step: 83, Cost: 0.550
Step: 84, Cost: 0.550


In [53]:
saver.save(sess,'./model/dnn.ckpt',global_step=global_step)

'./model/dnn.ckpt-84'

## Prediction and Accuracy

In [54]:
prediction=tf.argmax(model,1)
target=tf.argmax(Y,1)
print('Prediction value:', sess.run(prediction,feed_dict={X:x_data}))
print('Real value: ',sess.run(target,feed_dict={Y:y_data}))

Prediction value: [0 1 2 0 0 2]
Real value:  [0 1 2 0 0 2]


In [55]:
is_correct=tf.equal(prediction,target)
accuracy=tf.reduce_mean(tf.cast(is_correct,tf.float32))
print('Accuracy: %2f' %sess.run(accuracy*100,feed_dict={X:x_data,Y:y_data}))

Accuracy: 100.000000
